In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../data/test_dataset.tsv", delimiter="\t")
data.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,57809,"Listen, call off the butchers, and I'll tell you.",call out your butchers and I'll tell you.,0.761781,0.160000,0.005592,0.968063
1,132693,Who the fuck has been going through my stuff?!,who the hell was going through my stuff?,0.898699,0.127660,0.997644,0.452115
2,254505,She still might die . . .?,he can still die.,0.635970,0.333333,0.279049,0.989168
3,451186,"Yeah, that's the fucker's name.",that's what his name was.,0.652426,0.187500,0.999579,0.000055
4,191213,"I would take you on my shoulders, like, I'd st...",I'd take you on your shoulders... I'd tie you ...,0.790915,0.357143,0.961127,0.028628


In [3]:
data = data.drop("Unnamed: 0", axis=1)

In [4]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
model_name="t5-base"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained('../models')

/opt/conda/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [21]:
import torch
from tqdm import trange

In [31]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)
output = []
for i in trange(0, len(data.reference), 128):
    tokens = tokenizer(
        list(data.reference[i:i + 128]),
        padding="max_length",
        max_length=50,
        truncation=True,
        return_tensors="pt",
    ).to(device)

    out = model.generate(
        input_ids = tokens["input_ids"],
        attention_mask = tokens["attention_mask"],
        max_length= 50,
        num_return_sequences = 1
    )
    output += [tokenizer.decode(
        gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True
    ) for gen_id in out]
    
    
    
    

100%|██████████| 903/903 [07:21<00:00,  2.05it/s]


In [62]:
# !pip install sentence_transformers

In [35]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [41]:
embeddings1 = []
embeddings2 = []

embeddings1 = model.encode(list(data.reference), show_progress_bar=True, convert_to_tensor=True)
embeddings2 = model.encode(output, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/3612 [00:00<?, ?it/s]

Exception ignored in: <function tqdm.__del__ at 0x7fa620621160>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/tqdm/std.py", line 1149, in __del__
    self.close()
  File "/opt/conda/lib/python3.8/site-packages/tqdm/notebook.py", line 278, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


Batches:   0%|          | 0/3612 [00:00<?, ?it/s]

In [42]:
cosine_scores = []
for i in trange(len(embeddings1)):
    cosine_scores += util.cos_sim(embeddings1[i], embeddings2[i])

100%|██████████| 115555/115555 [00:11<00:00, 10144.84it/s]


In [56]:
average_similiraty_score = sum(cosine_scores) / len(embeddings1)
print(float(average_similiraty_score.detach()))

0.8497145771980286


In [63]:
from transformers import AutoModelForSequenceClassification

In [64]:
model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [74]:
def text2toxicity(text, aggregate=True):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

In [78]:
sum_toxicity = 0
for i in trange (len(data)):
    sum_toxicity += text2toxicity(data.reference[i], True)
print(sum_toxicity)


100%|██████████| 115555/115555 [02:57<00:00, 649.59it/s]

58475.971778956104


In [79]:
result_toxicity_average_before = sum_toxicity / len(data)
print(result_toxicity_average_before)

0.5060444963779681


In [80]:
sum_toxicity = 0
for i in trange (len(data)):
    sum_toxicity += text2toxicity(output[i], True)
print(sum_toxicity)

100%|██████████| 115555/115555 [02:53<00:00, 665.60it/s]

52378.66827116993


In [87]:
result_toxicity_average_before = sum_toxicity / len(output)
print(result_toxicity_average_before)

0.45327911618856764
